<a href="https://colab.research.google.com/github/dunkelweizen/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/Cai_Nowicki_assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.
- [ ] Watch Aaron's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [63]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [0]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [0]:
import numpy as np

In [0]:
true_false_columns = ['Chips', 'Beef', 'Pico', 'Guac', 'Cheese', 'Fries', 'Sour cream', 'Pork', 
                      'Chicken', 'Shrimp', 'Rice', 'Beans', 'Lettuce', 'Bell peper', 'Carrots',
                      'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 'Pineapple',
                      'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso', 'Egg', 'Mushroom',
                      'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini']

In [0]:
df[true_false_columns] = df[true_false_columns].replace(('x', 'X', np.NaN), (True, True, False))

In [68]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,False,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,True,True,True,True,True,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,California,2016-01-24,3.5,3.3,False,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,True,True,True,True,True,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Carnitas,2016-01-24,NaN,NaN,False,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,False,True,True,False,False,False,True,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Asada,2016-01-24,NaN,NaN,False,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,True,True,True,False,False,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,California,2016-01-27,4.0,3.8,True,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,True,True,False,True,True,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [70]:
df.median()

Yelp                4.000000
Google              4.200000
Cost                6.990000
Hunger              3.500000
Mass (g)          540.000000
Density (g/mL)      0.658099
Length             20.000000
Circum             22.000000
Volume              0.770000
Tortilla            3.500000
Temp                4.000000
Meat                3.800000
Fillings            3.500000
Meat:filling        4.000000
Uniformity          3.500000
Salsa               3.500000
Synergy             3.800000
Wrap                4.000000
Beef                0.000000
Pico                0.000000
Guac                0.000000
Cheese              0.000000
Fries               0.000000
Sour cream          0.000000
Pork                0.000000
Chicken             0.000000
Shrimp              0.000000
Rice                0.000000
Beans               0.000000
Lettuce             0.000000
Bell peper          0.000000
Carrots             0.000000
Cabbage             0.000000
Sauce               0.000000
Salsa.1       

In [0]:
#replace all NaN values with the median value for that column
df = df.fillna(df.median())

In [89]:
df['Unreliable'].unique()

array([False,  True])

In [0]:
df['Unreliable'] = df['Unreliable'].replace(('x', np.NaN), (True, False))

In [88]:
df['NonSD'].unique()

array([nan, 'x', 'X'], dtype=object)

In [0]:
df['NonSD'] = df['NonSD'].replace(('x', 'X', np.NaN), (True, True, False))

In [91]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,False,6.49,3.0,540.0,0.658099,20.0,22.0,0.77,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,False,False,True,True,True,True,True,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,California,2016-01-24,3.5,3.3,False,5.45,3.5,540.0,0.658099,20.0,22.0,0.77,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,False,False,True,True,True,True,True,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Carnitas,2016-01-24,4.0,4.2,False,4.85,1.5,540.0,0.658099,20.0,22.0,0.77,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,False,False,False,True,True,False,False,False,True,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Asada,2016-01-24,4.0,4.2,False,5.25,2.0,540.0,0.658099,20.0,22.0,0.77,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,False,False,True,True,True,False,False,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,California,2016-01-27,4.0,3.8,True,6.59,4.0,540.0,0.658099,20.0,22.0,0.77,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,False,False,True,True,False,True,True,False,False,False,False,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [0]:
train = df[(df['Date'] < '2017-01-01')]
validate = df[(df['Date'] >= '2017-01-01') & (df['Date'] < '2018-01-01')]
test = df[(df['Date'] >= '2018-01-01')]

In [93]:
print(train.shape)
print(validate.shape)
print(test.shape)

(298, 59)
(85, 59)
(38, 59)


In [94]:
target = 'Great'
y_train = train[target]
print('Percentage of Great \n', y_train.value_counts(normalize=True))

#guessing the majority class as the baseline
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)

Percentage of Great 
 False    0.590604
True     0.409396
Name: Great, dtype: float64


In [95]:
#testing the accuracy of the baseline model
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.5906040268456376

In [0]:
baseline_accuracy = accuracy_score(y_train, y_pred)

In [0]:
y_val = validate[target]

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
features = ['Hunger', 'Meat', 'Fillings']

From personal experience, when I'm really hungry everything tastes better!

In [0]:
X_train = train[features]
X_val = validate[features]

In [103]:
log_reg = LogisticRegression(solver='lbfgs')

log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [104]:
log_reg.predict([[4, 2, 2]])

array([False])

In [107]:
print('Validation Accuracy', log_reg.score(X_val, y_val))
print('Baseline Accuracy', baseline_accuracy)

Validation Accuracy 0.8941176470588236
Baseline Accuracy 0.5906040268456376


In [109]:
train.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great'],
      dtype='object')

In [0]:
features = ['Meat', 'Fillings', 'Meat:filling', 'Salsa', 'Synergy']
X_train = train[features]
X_val = validate[features]


In [0]:
def log_reg_score(data_train, data_val, features, target):
  X_train = data_train[features]
  X_val = data_val[features]
  y_train = data_train[target]
  y_val = data_val[target]
  log_reg = LogisticRegression(solver='lbfgs')
  log_reg.fit(X_train, y_train)
  print(log_reg.score(X_val, y_val))